# **Maxime's Notebook - Focus on Cycle GAN**

what is a cycle gan ?  
CycleGAN augmentation → Image dataset → Classification model (CNN, ResNet, etc.) → Soil type prediction

In [20]:
# Importation of libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torchvision
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
import tensorboard
from PIL import Image
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
from itertools import cycle

In [21]:
# Paths to data directories
original_dataset_dir = r"../data/Orignal-Dataset"
cyaug_dataset_dir = r"../data/CyAUG-Dataset"

In [22]:
# Create two versions of your dataset:
# Domain A: Original images (from your data folder) -> pictures inside original_dataset_dir
# Domain B: Same images with random augmentations applied

class SoilDataset(Dataset):
    """Dataset loader for soil images organized by soil type subdirectories"""
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.soil_types = []
        
        # Get all soil type subdirectories
        soil_type_dirs = [d for d in os.listdir(root_dir) 
                         if os.path.isdir(os.path.join(root_dir, d))]
        
        # Collect all image paths
        for soil_type in sorted(soil_type_dirs):
            soil_dir = os.path.join(root_dir, soil_type)
            for img_file in os.listdir(soil_dir):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.image_paths.append(os.path.join(soil_dir, img_file))
                    self.soil_types.append(soil_type)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        soil_type = self.soil_types[idx]
        
        # Read image
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, soil_type

# Define transformations for Domain B (augmented version)
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256, 256)),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.GaussianBlur(kernel_size=3),
    torchvision.transforms.ToTensor(),
])

transforms_original = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256, 256)),
    torchvision.transforms.ToTensor(),
])


In [23]:
# Load datasets with proper structure handling
print("Loading datasets...")
domain_A = SoilDataset(original_dataset_dir, transform=transforms_original)
domain_B = SoilDataset(original_dataset_dir, transform=transforms)

print(f"Domain A (Original): {len(domain_A)} images")
print(f"Domain B (Augmented): {len(domain_B)} images")

# Create data loaders
batch_size = 7
dataloader_A = DataLoader(domain_A, batch_size=batch_size, shuffle=True)
dataloader_B = DataLoader(domain_B, batch_size=batch_size, shuffle=True)

# Check if data loads correctly
sample_img_A, soil_type = next(iter(dataloader_A))
print(f"\nSample batch shape (Domain A): {sample_img_A.shape}")
print(f"Soil types in batch: {soil_type}")


Loading datasets...
Domain A (Original): 1188 images
Domain B (Augmented): 1188 images

Sample batch shape (Domain A): torch.Size([7, 3, 256, 256])
Soil types in batch: ('Mountain_Soil', 'Black_Soil', 'Arid_Soil', 'Laterite_Soil', 'Mountain_Soil', 'Arid_Soil', 'Black_Soil')


In [24]:
import torch.nn as nn
import torch.nn.functional as F
from itertools import cycle

# Define Generator (ResNet-based)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels)
        )
    
    def forward(self, x):
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_residual_blocks=6):
        super(Generator, self).__init__()
        
        # Initial convolution layer
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_channels, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]
        
        # Downsampling
        model += [
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True)
        ]
        
        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(256)]
        
        # Upsampling
        model += [
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]
        
        # Final convolution layer
        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(64, out_channels, 7),
            nn.Tanh()
        ]
        
        self.model = nn.Sequential(*model)
    
    def forward(self, x):
        return self.model(x)

# Define Discriminator (PatchGAN)
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()
        
        model = [
            nn.Conv2d(in_channels, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        ]
        
        model += [
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        ]
        
        model += [
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        ]
        
        model += [
            nn.Conv2d(256, 512, 4, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        ]
        
        model += [nn.Conv2d(512, 1, 4, padding=1)]
        
        self.model = nn.Sequential(*model)
    
    def forward(self, x):
        return self.model(x)

print("Generator and Discriminator classes defined successfully!")


Generator and Discriminator classes defined successfully!


In [25]:
# Initialize models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

generator_A2B = Generator().to(device)
generator_B2A = Generator().to(device)
discriminator_A = Discriminator().to(device)
discriminator_B = Discriminator().to(device)

# Loss functions
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

# Optimizers
lr = 0.0002
beta1 = 0.5
optimizer_G = torch.optim.Adam(
    list(generator_A2B.parameters()) + list(generator_B2A.parameters()),
    lr=lr, betas=(beta1, 0.999)
)
optimizer_D_A = torch.optim.Adam(discriminator_A.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D_B = torch.optim.Adam(discriminator_B.parameters(), lr=lr, betas=(beta1, 0.999))

print("Models and optimizers initialized!")


Using device: cpu
Models and optimizers initialized!


In [ ]:
# Training loop
num_epochs = 10
lambda_cycle = 10.0  # Weight for cycle consistency loss
lambda_identity = 5.0  # Weight for identity loss

print("Starting CycleGAN training...")
print(f"Epochs: {num_epochs}, Batch size: {batch_size}")

for epoch in range(num_epochs):
    epoch_loss_G = 0
    epoch_loss_D = 0
    
    # Use itertools.cycle to handle different dataset sizes
    for (real_A, _), (real_B, _) in zip(dataloader_A, cycle(dataloader_B)):
        real_A = real_A.to(device)
        real_B = real_B.to(device)
        
        # --- Generator forward pass ---
        # Generate fake images
        fake_B = generator_A2B(real_A)
        fake_A = generator_B2A(real_B)
        
        # Cycle consistency
        cycle_A = generator_B2A(fake_B)
        cycle_B = generator_A2B(fake_A)
        
        # Identity mapping
        identity_A = generator_B2A(real_A)
        identity_B = generator_A2B(real_B)
        
        # --- Discriminator losses ---
        optimizer_D_A.zero_grad()
        optimizer_D_B.zero_grad()
        
        # Real/Fake discrimination
        real_A_pred = discriminator_A(real_A)
        fake_A_pred = discriminator_A(fake_A.detach())
        real_B_pred = discriminator_B(real_B)
        fake_B_pred = discriminator_B(fake_B.detach())
        
        # Loss computation
        loss_D_A = criterion_GAN(real_A_pred, torch.ones_like(real_A_pred)) + \
                   criterion_GAN(fake_A_pred, torch.zeros_like(fake_A_pred))
        loss_D_B = criterion_GAN(real_B_pred, torch.ones_like(real_B_pred)) + \
                   criterion_GAN(fake_B_pred, torch.zeros_like(fake_B_pred))
        
        loss_D = loss_D_A + loss_D_B
        loss_D.backward()
        optimizer_D_A.step()
        optimizer_D_B.step()
        
        # --- Generator losses ---
        optimizer_G.zero_grad()
        
        # GAN loss
        fake_A_pred = discriminator_A(fake_A)
        fake_B_pred = discriminator_B(fake_B)
        loss_GAN_A2B = criterion_GAN(fake_B_pred, torch.ones_like(fake_B_pred))
        loss_GAN_B2A = criterion_GAN(fake_A_pred, torch.ones_like(fake_A_pred))
        
        # Cycle loss
        loss_cycle_A = criterion_cycle(cycle_A, real_A)
        loss_cycle_B = criterion_cycle(cycle_B, real_B)
        loss_cycle = loss_cycle_A + loss_cycle_B
        
        # Identity loss
        loss_identity_A = criterion_identity(identity_A, real_A)
        loss_identity_B = criterion_identity(identity_B, real_B)
        loss_identity = loss_identity_A + loss_identity_B
        
        # Total generator loss
        loss_G = loss_GAN_A2B + loss_GAN_B2A + lambda_cycle * loss_cycle + lambda_identity * loss_identity
        loss_G.backward()
        optimizer_G.step()
        
        epoch_loss_G += loss_G.item()
        epoch_loss_D += loss_D.item()
    
    avg_loss_G = epoch_loss_G / len(dataloader_A)
    avg_loss_D = epoch_loss_D / len(dataloader_A)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - G Loss: {avg_loss_G:.4f}, D Loss: {avg_loss_D:.4f}")

print("\nTraining completed!")


Starting CycleGAN training...
Epochs: 10, Batch size: 7


In [ ]:
# Generate augmented images using trained generator
output_dir = os.path.join(original_dataset_dir, "..", "CycleGAN_Augmented")
os.makedirs(output_dir, exist_ok=True)

print(f"Generating augmented images to: {output_dir}")

generator_A2B.eval()
generator_B2A.eval()

with torch.no_grad():
    for soil_type in os.listdir(original_dataset_dir):
        soil_type_path = os.path.join(original_dataset_dir, soil_type)
        
        if not os.path.isdir(soil_type_path):
            continue
        
        # Create output directory for this soil type
        output_soil_dir = os.path.join(output_dir, soil_type)
        os.makedirs(output_soil_dir, exist_ok=True)
        
        # Process each image
        for img_file in os.listdir(soil_type_path):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            img_path = os.path.join(soil_type_path, img_file)
            image = Image.open(img_path).convert('RGB')
            image_tensor = transforms_original(image).unsqueeze(0).to(device)
            
            # Generate augmented version
            augmented = generator_A2B(image_tensor)
            
            # Convert back to image
            augmented_img = augmented.squeeze(0).cpu()
            augmented_img = (augmented_img * 0.5 + 0.5).clamp(0, 1)  # Denormalize
            augmented_pil = torchvision.transforms.ToPILImage()(augmented_img)
            
            # Save augmented image
            output_path = os.path.join(output_soil_dir, f"augmented_{img_file}")
            augmented_pil.save(output_path)
        
        print(f"✓ Processed {soil_type}")

print(f"\n✓ All augmented images saved to: {output_dir}")
